In [26]:
import os
import shutil
from IPython.display import clear_output

In [21]:
class DTU():
    def __init__(self):
        self.source=""
        self.destination=""
        self.mediafolder="/Volumes"
        self.drives=os.listdir(self.mediafolder)
        
    def showMenu(self):
        print("-----MENU-----")
        for i in range(1,6):
            if i == 1:
                print("[1] Show all sources\n")

            elif i== 2:
                print("[2] Select source\n")

            elif i== 3:
                print("[3] Select Destination\n")

            elif i== 4:
                print("[4] Transfer content\n")

            elif i== 5:
                print("[5] Exit\n")

            else:
                break

        choice = int(input("Please select an option: "))
        print("--------------")
        return choice

    def choiceSelected(self,choice):
        try:
            
            if choice == 1:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)
                return

            elif choice == 2:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)

                select = int(input("Select a source: "))
                self.source=self.drives[select-1]
                print("\nSource selected!\n")
                return

            elif choice == 3:
                print("Availabe destinations:\n")
                for i,source in enumerate(self.drives):
                    if source==self.source:
                        continue
                    print(i+1,source)

                select = int(input("Select a destination: "))
                self.destination=self.drives[select-1]
                print("\nDestination selected!\n")
                return

            elif choice == 4:
                print("")

            elif choice == 5:
                pass

        except Exception as e:
            print("ERROR: ",e)

In [22]:
print("Welcome to DTU")
dtu=DTU()

Welcome to DTU


In [28]:
while(True):
    ch=dtu.showMenu()
    
    #Exit condition
    if ch not in [1,2,3,4]:
        print("Bye for now!")
        break
    
    #On choice selected
    dtu.choiceSelected(ch)
    input("press [ENTER] to continue")
    clear_output()

-----MENU-----
[1] Show all sources

[2] Select source

[3] Select Destination

[4] Transfer content

[5] Exit

Please select an option: 7
--------------
Bye for now!
